# Index

- **Python Libraries**
- **Task 2**
    - *Reading Preprocessed Datasets*
    - *Exercise 1*
    - *Exercise 2*
        - *Balancing Strategy Definition*
        - *Exercise 2.1*
        - *Exercise 2.2*
        - *Exercise 2.3*
    - *Exercise 3 - Concentration*
    - *Writing Balanced Datasets*


In [100]:
import pandas as pd
import numpy as np
import datetime as dt

# Task 2

## Reading Preprocessed Datasets

In [101]:
read_data_path  = "../../data/dataset_preprocessed"

ap   = pd.read_csv(read_data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso','label'])
diag = pd.read_csv(read_data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(read_data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei   = pd.read_csv(read_data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf  = pd.read_csv(read_data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(read_data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd  = pd.read_csv(read_data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [102]:
print(ap.shape)
ap.head()

(1555, 13)


,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso,label
0,1,5,M,1986.0,5,2.0,2.0,9.0,NaN,1942,1990.0,2014.0,1
1,1,65,M,1997.0,5,2.0,1.0,9.0,NaN,1942,1997.0,2018.0,1
2,1,74,M,1995.0,5,2.0,2.0,9.0,NaN,1930,1996.0,2013.0,0
3,1,143,M,1991.0,5,2.0,NaN,9.0,NaN,1934,1991.0,2015.0,0
4,1,173,M,1988.0,5,2.0,2.0,9.0,NaN,1939,1988.0,2012.0,1


In [103]:
print(diag.shape)
diag.head()

(106999, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,1997-12-01,AMD247,410
1,1,5,2000-06-01,AMD247,434.01
2,1,5,2000-06-01,AMD247,434.91
3,1,5,2000-12-02,AMD049,S
4,1,5,2000-12-02,AMD247,36.10


In [104]:
print(elp.shape)
elp.head()

(312389, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.000000
1,1,5,2005-01-18,AMD002,76.000000
2,1,5,2005-01-18,AMD004,90.232558
3,1,5,2005-01-18,AMD005,49.600000
4,1,5,2005-06-06,AMD007,138.225058


In [105]:
print(ei.shape)
ei.head()

(15114, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [106]:
print(pdf.shape)
pdf.head()

(94228, 7)


,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,1,5,2005-01-18,A10BA02,2.0,1.0,METFORAL*50CPR RIV 500mg
1,1,5,2005-01-18,A10BA02,2.0,3.0,METFORAL*50CPR RIV 500mg
2,1,5,2005-01-18,A10BA02,2.0,5.0,METFORAL*50CPR RIV 500mg
3,1,5,2005-01-18,A10BB12,1.0,3.0,AMARYL*30CPR 2mg
4,1,5,2005-06-21,A10BA02,2.0,1.0,METFORAL*50CPR RIV 500mg


In [107]:
print(pdnf.shape)
pdnf.head()

(9954, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,65,2014-12-19,AMD228,S


In [108]:
print(pnd.shape)
pnd.head()

(105297, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


## Exercise 1

*Class imbalance #1* - not all patients will have a cardiovascular event within the stabilised six-month period. Thus, we would expect that the class distribution is highly imbalanced. 

- For each patient $p_i$ such that $y(p_i)=0$, eliminate the last six months of history to avoid giving the model prediction hints into the future. 
- For each patient $p_i$ such that $y(p_i)=1$, create $m$ copies $\lbrace p_i^1, \cdots, p_i^m \rbrace$ such that all the cardiovascular events in the last six months are eliminated, and the other events are shuffled and cancelled at random.

In this way, you have a sort of balancing criterion (i.e., up-sampling the minority class).

First we divide the **ap** dataframe into:
- **ap_class0** each patient $p_i$ has $y(p_i)=0$
- **ap_class1** each patient $p_i$ has $y(p_i)=1$

In [109]:
print(f"ap has size        => {ap.shape}")
ap_class0 = ap[ap.label == 0]
print(f"ap_class0 has size => {ap_class0.shape}")
ap_class1 = ap[ap.label == 1]
print(f"ap_class1 has size => {ap_class1.shape}")


ap has size        => (1555, 13)
ap_class0 has size => (868, 13)
ap_class1 has size => (687, 13)


And now the rest of the dataframes are also divided into **df_class0** and **df_class1** following the same criteria:

In [110]:
diag_class0 = pd.merge(diag, ap_class0, on=['idcentro','idana'])[diag.columns]
elp_class0  = pd.merge(elp, ap_class0, on=['idcentro','idana'])[elp.columns]
ei_class0   = pd.merge(ei, ap_class0, on=['idcentro','idana'])[ei.columns]
pdf_class0  = pd.merge(pdf, ap_class0, on=['idcentro','idana'])[pdf.columns]
pdnf_class0 = pd.merge(pdnf, ap_class0, on=['idcentro','idana'])[pdnf.columns]
pnd_class0  = pd.merge(pnd, ap_class0, on=['idcentro','idana'])[pnd.columns]

diag_class1 = pd.merge(diag, ap_class1, on=['idcentro','idana'])[diag.columns]
elp_class1  = pd.merge(elp, ap_class1, on=['idcentro','idana'])[elp.columns]
ei_class1   = pd.merge(ei, ap_class1, on=['idcentro','idana'])[ei.columns]
pdf_class1  = pd.merge(pdf, ap_class1, on=['idcentro','idana'])[pdf.columns]
pdnf_class1 = pd.merge(pdnf, ap_class1, on=['idcentro','idana'])[pdnf.columns]
pnd_class1  = pd.merge(pnd, ap_class1, on=['idcentro','idana'])[pnd.columns]

We define a function that eliminates the events during the last 6 months for each patient

In [111]:
def dropLastSixMonths(df:pd.DataFrame, patients: pd.DataFrame) -> pd.DataFrame:

    newDF = pd.DataFrame(columns=df.columns)

    # We iterate for each patient on patients
    for p in patients.itertuples():
        # Take the cardiovascular events of the patient p
        aux = df[(df.idcentro==p.idcentro) & (df.idana==p.idana)]

        # Calculate the last valid data
        aux = aux.sort_values('data', ascending=False)
        lastData = next(aux.itertuples()).data
        lastData = dt.date.fromisoformat(lastData)
        limitData = lastData - dt.timedelta(days=30*6)

        # Take only the events before this limit data
        aux = aux[aux.data < limitData.strftime("%Y-%m-%d")]
        
        # We concatenate this with the new DataFrame
        newDF = pd.concat([newDF, aux])

    return newDF

Now we execute the function to every dataframe in our preprocessed dataset for the **ap_class0** patients:

In [112]:
aux = diag_class0.shape[0]
diag_class0 = dropLastSixMonths(diag_class0, ap_class0)
print(f"Diag_class0: {aux} => {diag_class0.shape[0]}")

aux = elp_class0.shape[0]
elp_class0 = dropLastSixMonths(elp_class0, ap_class0)
print(f"Elp_class0: {aux} => {elp_class0.shape[0]}")

aux = ei_class0.shape[0]
ei_class0 = dropLastSixMonths(ei_class0, ap_class0)
print(f"Ei_class0: {aux} => {ei_class0.shape[0]}")

aux = pdf_class0.shape[0]
pdf_class0 = dropLastSixMonths(pdf_class0, ap_class0)
print(f"Pdf_class0: {aux} => {pdf_class0.shape[0]}")

aux = pdnf_class0.shape[0]
pdnf_class0 = dropLastSixMonths(pdnf_class0, ap_class0)
print(f"Pdnf_class0: {aux} => {pdnf_class0.shape[0]}")

aux = pnd_class0.shape[0]
pnd_class0 = dropLastSixMonths(pnd_class0, ap_class0)
print(f"Pnd_class0: {aux} => {pnd_class0.shape[0]}")

Diag_class0: 49914 => 44071
Elp_class0: 168320 => 146637
Ei_class0: 7825 => 6291
Pdf_class0: 49264 => 44101
Pdnf_class0: 5718 => 4299
Pnd_class0: 51834 => 45656


Now for the **ap_class1** patients:

In [113]:
aux = diag_class1.shape[0]
diag_class1 = dropLastSixMonths(diag_class1, ap_class1)
print(f"Diag_class1: {aux} => {diag_class1.shape[0]}")

aux = elp_class1.shape[0]
elp_class1 = dropLastSixMonths(elp_class1, ap_class1)
print(f"Elp_class1: {aux} => {elp_class1.shape[0]}")

aux = ei_class1.shape[0]
ei_class1 = dropLastSixMonths(ei_class1, ap_class1)
print(f"Ei_class1: {aux} => {ei_class1.shape[0]}")

aux = pdf_class1.shape[0]
pdf_class1 = dropLastSixMonths(pdf_class1, ap_class1)
print(f"Pdf_class1: {aux} => {pdf_class1.shape[0]}")

aux = pdnf_class1.shape[0]
pdnf_class1 = dropLastSixMonths(pdnf_class1, ap_class1)
print(f"Pdnf_class1: {aux} => {pdnf_class1.shape[0]}")

aux = pnd_class1.shape[0]
pnd_class1 = dropLastSixMonths(pnd_class1, ap_class1)
print(f"Pnd_class1: {aux} => {pnd_class1.shape[0]}")

Diag_class1: 57085 => 49112
Elp_class1: 144069 => 126055
Ei_class1: 7289 => 5822
Pdf_class1: 44964 => 40007
Pdnf_class1: 4236 => 3153
Pnd_class1: 53463 => 47128


At this point, we are done with the **ap_class0** patients, so we focus on the **ap_class1** ones. We now have to create $m$ (= 10 for example) copies of each patient $p_i$ such that $y(p_i)=1$, and we shuffle or eliminate cardiovascular events randomly just to *oversample* the dataset:

In [ ]:
def newMCopies(patients:pd.DataFrame, copies:int) -> pd.DataFrame:

    newPatients = pd.DataFrame(columns=patients.columns)
    
    for p in patients.itertuples():
        # TO DO - Habría que crear en la tabla ap_class1 una nueva columna llamada idcopy para que cada paciente se 
        # identificase como (idcentro,idana,idcopy). Para cada idcopy fijado, todos los pacientes son iguales

    return newPatients

In [ ]:
def oversamplingClass1(df:pd.DataFrame, patients:pd.DataFrame, m:int) -> pd.DataFrame:
    newAp_class1 = newMCopies(patients=ap_class1, copies=m)
    for p in patients.itertuples():
        # TO DO - aquí habría que crear también para cada evento una columna idcopy para asociarlo al paciente concreto, y ya pues
        # investigar como hacer el shuffle aleatoriamente para cada paciente.
    return None

In [ ]:
M = 10
diag_class1 = oversamplingClass1(df=diag_class1, patients=ap_class1, m=M)

## Exercise 2

*Class Imbalance #2* - Action item *#1* isn't going to be sufficient for balancing purposes. Propose your balancing strategy - possibly an advanced approach - and evaluate: 
- **vanilla-LTSM** (*Exercise 2.1*)
- **T-LTSM** (*Exercise 2.2*)
- **PubMedBert** (*Exercise 2.3*) 

on the balanced version of the dataset.

### Balancing Strategy Definition

### Exercise 2.1

### Exercise 2.2

### Exercise 2.3